In [1]:
from tools.preprocess import LoadDataset

import pandas as pd
import numpy as np

from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from catboost import Pool,CatBoostClassifier

train_path = "../data/train.csv"
test_path = "../data/valid.csv"
valid_path = "../data/valid.csv"

# do with train, test, valid
data = LoadDataset(train_path,
                   test_path, 
                   valid_path)

data.nan_processing({"ALL": "DROP COLUMN"})
data.sampling("UNDER")
data.scaling({"ALL": "STANDARD"})
data.encoding({"ALL": "LABEL"})
data.feature_engineering("PCA PER PROCESS", {"min_proba": 0.9999})
print(data.train.x.shape)

Missing Value Processing...

(before processing)The number of nan
- train: 59028
- valid: 13149

  1. Processing <ALL> Columns with 'DROP COLUMN'

Finish!
(after processing)The number of nan
- train: 0
- valid: 0

Sampling with UNDER...

(before sampling)Value count
target
0    31156
1     2000
Name: count, dtype: int64 

Finish!
(after sampling)Value count
target
0    2000
1    2000
Name: count, dtype: int64

Scaling our dataset...

  1. Scaling <ALL> Columns with 'STANDARD'

Finish! (the number of columns: 130)

Encoding our dataset...

  1. Encoding <ALL> Columns with 'LABEL'

Finish! (the number of columns: 12)

Feature Engineering with PCA PER PROCESS...

  Execute PCA PER PROCESS with minimum probability 0.9999

  Stage 'Dam': the number of target cols is '68'
  => 9_components with Explained Variance Ratio '0.9999086628211619'

  Stage 'Fill1': the number of target cols is '32'
  => 8_components with Explained Variance Ratio '0.9999071092471651'

  Stage 'Fill2': the number of t

In [2]:
from chempfn import ChemPFN

classifier = ChemPFN(max_iters=100)
classifier.fit(data.train.x, data.train.y)
y_eval = classifier.predict(data.valid.x)

# print('Accuracy', accuracy_score(y_test, y_eval))

c:\Users\abrah\anaconda3\envs\Capstone\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

In [49]:
p_eval

array([0.51790065, 0.5787135 , 0.5392853 , ..., 0.5376424 , 0.5173196 ,
       0.5133485 ], dtype=float32)

In [50]:
p_eval > 0.6

array([False, False, False, ..., False, False, False])

In [55]:
pd.DataFrame(y_eval).value_counts()

0    4529
1    2821
Name: count, dtype: int64

In [48]:
answer = data.valid.y.replace({0:"Normal", 1:"AbNormal"})
f1 = f1_score(answer, test_pred, pos_label = "AbNormal")
print(f1)

0.24423337856173677
